In [17]:
import json
import re

import numpy as np
import pandas as pd

In [18]:
METRICS = [
    "agent_total_tokens",
    "agent_completion_tokens",
    "agent_prompt_tokens",
    "agent_llm_calls",
    "num_user_turns",
    "num_tool_calls",
    "agent_active_time",
    "duration",
    "agent_cost",
]

STAT_COLS = ["n", "min", "mean", "p50", "p90", "p95", "p99", "max", "std"]


def _cat(task_id):
    m = re.match(r"\[([^\]]+)\]", task_id)
    return m.group(1) if m else "unknown"


def _n_actions(task):
    return len((task.get("evaluation_criteria") or {}).get("actions") or [])


def _success(sim):
    ri = sim.get("reward_info")
    return ri is not None and abs(ri.get("reward", 0) - 1.0) < 1e-6


def _reward(sim):
    ri = sim.get("reward_info")
    return ri.get("reward", 0) if ri else 0.0


def _stats_series(vals):
    if not vals:
        return {k: np.nan for k in STAT_COLS}
    a = np.array(vals, dtype=float)
    return {
        "n": len(a),
        "min": np.min(a),
        "mean": np.mean(a),
        "p50": np.median(a),
        "p90": np.percentile(a, 90),
        "p95": np.percentile(a, 95),
        "p99": np.percentile(a, 99),
        "max": np.max(a),
        "std": np.std(a, ddof=1) if len(a) > 1 else 0.0,
    }


def _flatten_rt(series):
    flat = []
    for rt in series.dropna():
        if isinstance(rt, list):
            flat.extend(rt)
    return flat


def _compute_stats_df(df, metrics=None):
    if metrics is None:
        metrics = [c for c in METRICS if c in df.columns]
    rows = []
    for m in metrics:
        s = _stats_series(df[m].dropna().tolist())
        s["metric"] = m
        rows.append(s)
    if "agent_response_times" in df.columns:
        flat = _flatten_rt(df["agent_response_times"])
        if flat:
            s = _stats_series(flat)
            s["metric"] = "agent_response_time"
            rows.append(s)
    return pd.DataFrame(rows).set_index("metric")[STAT_COLS]


def _grouped_stats_df(df, group_col, metrics=None):
    if metrics is None:
        metrics = [c for c in METRICS if c in df.columns]
    rows = []
    for gval, gdf in df.groupby(group_col):
        for m in metrics:
            s = _stats_series(gdf[m].dropna().tolist())
            s["group"] = gval
            s["metric"] = m
            rows.append(s)
        if "agent_response_times" in gdf.columns:
            flat = _flatten_rt(gdf["agent_response_times"])
            if flat:
                s = _stats_series(flat)
                s["group"] = gval
                s["metric"] = "agent_response_time"
                rows.append(s)
        if "success" in gdf.columns:
            n = len(gdf)
            p = gdf["success"].sum()
            rows.append(
                {
                    "group": gval,
                    "metric": "success_rate",
                    "n": n,
                    "mean": p / n if n else np.nan,
                    **{k: np.nan for k in STAT_COLS if k not in ["n", "mean"]},
                }
            )
    out = pd.DataFrame(rows).set_index(["group", "metric"])
    return out[STAT_COLS]


class RunAnalysis:
    def __init__(self, info: dict, sims: pd.DataFrame):
        self.info = info
        self.sims = sims

    def __repr__(self):
        ai = self.info.get("agent_info", {})
        domain = self.info.get("environment_info", {}).get("domain_name", "?")
        n = len(self.sims)
        p = int(self.sims["success"].sum()) if "success" in self.sims else "?"
        return f"RunAnalysis({ai.get('llm', '?')}, domain={domain}, n={n}, pass={p})"

    def overall(self):
        return _compute_stats_df(self.sims)

    def by(self, col):
        return _grouped_stats_df(self.sims, col)

    def by_category(self):
        return self.by("category")

    def by_outcome(self):
        return self.by("success")

    def by_actions(self):
        return self.by("n_expected_actions")

    def by_termination(self):
        return self.by("termination_reason")


def load_run(path) -> RunAnalysis:
    with open(path) as f:
        data = json.load(f)
    info = data.get("info", {})
    task_map = {t["id"]: t for t in data.get("tasks", [])}
    rows = []
    for sim in data.get("simulations", []):
        task = task_map.get(sim["task_id"], {})
        row = {
            "task_id": sim["task_id"],
            "category": _cat(sim["task_id"]),
            "n_expected_actions": _n_actions(task),
            "success": _success(sim),
            "reward": _reward(sim),
            "termination_reason": sim.get("termination_reason"),
            "trial": sim.get("trial"),
            "seed": sim.get("seed"),
            "agent_response_times": sim.get("agent_response_times"),
        }
        for m in METRICS:
            row[m] = sim.get(m)
        rows.append(row)
    return RunAnalysis(info, pd.DataFrame(rows))

In [19]:
run = load_run(
    "/Users/areremkin/Code/tau2-bench/data/simulations/2026-02-09T23:10:40.625891_telecom_mas_2_gpt-oss-120b_user_simulator_gpt-oss-120b.json"
)

In [20]:
run.sims

,task_id,category,n_expected_actions,success,reward,termination_reason,trial,seed,agent_response_times,agent_total_tokens,agent_completion_tokens,agent_prompt_tokens,agent_llm_calls,num_user_turns,num_tool_calls,agent_active_time,duration,agent_cost
0,[mobile_data_issue]bad_network_preference|bad_...,mobile_data_issue,3,False,0.0,user_stop,0,626729,"[1.4706675000488758, 2.1628092501778156, 1.746...",36488,543,35945,10,5,2,8.192862,13.801299,0.000773
1,[mobile_data_issue]bad_vpn|data_saver_mode_on|...,mobile_data_issue,3,False,0.0,user_stop,0,626729,"[1.6321470830589533, 2.134185832925141, 1.7710...",36550,640,35910,10,5,2,8.936691,14.163152,0.000782
2,[mobile_data_issue]data_usage_exceeded|user_ab...,mobile_data_issue,2,False,0.0,user_stop,0,626729,"[2.726437416858971, 2.759948165854439, 2.11877...",42464,594,41870,11,5,3,9.972858,15.302671,0.000897
3,[mobile_data_issue]airplane_mode_on|data_saver...,mobile_data_issue,3,False,0.0,user_stop,0,626729,"[2.5679715420119464, 2.1783245000988245, 1.434...",49216,885,48331,13,6,3,12.644782,18.098548,0.001055
4,[mobile_data_issue]airplane_mode_on|bad_networ...,mobile_data_issue,5,False,0.0,user_stop,0,626729,"[1.409363916143775, 2.0630275420844555, 1.4915...",50059,927,49132,14,7,2,13.135080,21.872907,0.001075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,[mms_issue]airplane_mode_on|bad_network_prefer...,mms_issue,10,True,1.0,user_stop,0,626729,"[10.567182707833126, 2.3961115840356797, 8.705...",235030,9072,225958,39,44,46,72.771561,120.675686,0.005426
110,[mms_issue]airplane_mode_on|bad_network_prefer...,mms_issue,12,True,1.0,user_stop,0,626729,"[2.201849792152643, 10.014923583017662, 2.9710...",210644,8060,202584,36,35,34,62.720344,96.909001,0.004858
111,[mms_issue]airplane_mode_on|bad_network_prefer...,mms_issue,11,False,0.0,user_stop,0,626729,"[2.0883687089663, 7.824182708980516, 3.1526584...",312130,13526,298604,46,39,33,94.435746,150.350789,0.007325
112,[mms_issue]airplane_mode_on|bad_network_prefer...,mms_issue,11,True,1.0,user_stop,0,626729,"[1.7083764169365168, 15.809256250038743, 9.352...",332372,13707,318665,48,43,45,91.715043,139.496229,0.007744


In [21]:
run.overall()

,n,min,mean,p50,p90,p95,p99,max,std
metric,,,,,,,,,
agent_total_tokens,114,24873.000000,110600.763158,86658.000000,198085.000000,240992.950000,329740.540000,725204.000000,88859.522872
agent_completion_tokens,114,423.000000,3654.921053,2583.500000,7812.000000,9121.700000,13683.470000,21297.000000,3355.613013
agent_prompt_tokens,114,24286.000000,106945.842105,83518.500000,190564.000000,230837.000000,316057.070000,703907.000000,85630.007408
agent_llm_calls,114,6.000000,21.517544,18.000000,35.700000,39.700000,47.740000,81.000000,11.541562
num_user_turns,114,2.000000,17.359649,15.000000,32.000000,36.050000,43.870000,62.000000,11.718020
num_tool_calls,114,1.000000,17.192982,16.000000,32.700000,38.050000,45.000000,46.000000,12.135574
agent_active_time,114,6.006979,34.828130,26.773434,68.294233,82.718282,103.647080,147.067059,25.905697
duration,114,8.382476,55.118449,43.331923,100.134120,124.456193,150.326295,236.958648,40.291259
agent_cost,114,0.000539,0.002504,0.001987,0.004532,0.005632,0.007689,0.016208,0.002037


In [22]:
run.by_category()

n           min           mean  \
group             metric                                                      
mms_issue         agent_total_tokens        49  35964.000000  149026.836735   
                  agent_completion_tokens   49    560.000000    5166.122449   
                  agent_prompt_tokens       49  35404.000000  143860.714286   
                  agent_llm_calls           49     10.000000      26.408163   
                  num_user_turns            49      5.000000      23.265306   
                  num_tool_calls            49      1.000000      23.142857   
                  agent_active_time         49      8.481020      46.205428   
                  duration                  49     12.566665      73.424668   
                  agent_cost                49      0.000764       0.003394   
                  agent_response_time      416      0.930501       5.397368   
                  success_rate              49           NaN       0.346939   
mobile_data_issue agent_total_tokens        36  35801.000000   83621.805556   
                  agent_completion_tokens   36    423.000000    2380.861111   
                  agent_prompt_tokens       36  35378.000000   81240.944444   
                  agent_llm_calls           36      9.000000      17.694444   
                  num_user_turns            36      4.000000      13.750000   
                  num_tool_calls            36      1.000000      13.027778   
                  agent_active_time         36      7.013599      25.307077   
                  duration                  36     11.572824      40.846879   
                  agent_cost                36      0.000750       0.001863   
                  agent_response_time      209      1.053485       4.356050   
                  success_rate              36           NaN       0.527778   
service_issue     agent_total_tokens        29  24873.000000   79165.068966   
                  agent_completion_tokens   29    525.000000    2683.103448   
                  agent_prompt_tokens       29  24286.000000   76481.965517   
                  agent_llm_calls           29      6.000000      18.000000   
                  num_user_turns            29      2.000000      11.862069   
                  num_tool_calls            29      4.000000      12.310345   
                  agent_active_time         29      6.006979      27.423660   
                  duration                  29      8.382476      41.903685   
                  agent_cost                29      0.000539       0.001798   
                  agent_response_time      160      1.126376       4.970274   
                  success_rate              29           NaN       0.793103   

                                                     p50            p90  \
group             metric                                                  
mms_issue         agent_total_tokens       139954.000000  238437.400000   
                  agent_completion_tokens    4588.000000    9420.600000   
                  agent_prompt_tokens      135366.000000  228746.000000   
                  agent_llm_calls              27.000000      39.400000   
                  num_user_turns               26.000000      35.600000   
                  num_tool_calls               26.000000      37.600000   
                  agent_active_time            45.777467      81.385310   
                  duration                     76.971283     122.835976   
                  agent_cost                    0.003205       0.005544   
                  agent_response_time           4.022030      10.572514   
                  success_rate                       NaN            NaN   
mobile_data_issue agent_total_tokens        66496.000000  142236.500000   
                  agent_completion_tokens    1737.000000    5128.000000   
                  agent_prompt_tokens       64597.500000  137216.000000   
                  agent_llm_calls              15.000000      26.500000   
    